<a href="https://colab.research.google.com/github/Somvit09/Test-LLMS/blob/master/SAM2_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -r requirements.txt

  Using cached segment_anything-1.0-py3-none-any.whl


In [31]:
!pip install numpy opencv-python torch torchvision python-dotenv fastapi uvicorn

In [32]:
!pip install requests

In [33]:
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator
import numpy as np
import cv2, os, torch, requests
from dotenv import load_dotenv
from enum import Enum

In [34]:
load_dotenv()
BASE_DIR = os.getcwd()
print(BASE_DIR)
CONFIG_PATH = os.path.join(BASE_DIR, "configs")
os.makedirs(CONFIG_PATH, exist_ok=True)
print(CONFIG_PATH)

/content
/content/configs


In [35]:
class ModelType(Enum):
    VIT_H = "vit_h"
    VIT_B = "vit_b"
    VIT = "vit_l"

In [36]:
class SAM2Annotator:
    def __init__(self):
        """
        Initialize the SAM2 model with the given model checkpoint path.
        """
        self.__MODEL_TYPE = os.getenv("MODEL_TYPE")
        self.__CHECKPOINT = os.getenv("CHECKPOINT")
        print("MODEL_TYPE", self.__MODEL_TYPE)
        print("CHECKPOINT", self.__CHECKPOINT)
        # empty cache
        torch.cuda.empty_cache()

        # download pth file
        self.download_configs()

        # initialize model
        self.__DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.__model = sam_model_registry[self.__MODEL_TYPE](checkpoint=self.download_path)
        self.__model.to(device=self.__DEVICE)
        self.mask_generator = SamAutomaticMaskGenerator(self.__model)
        self.predictor = SamPredictor(self.__model)

    def download_configs(self):
        VIT_H_PTH_URL: str = os.getenv("VIT_H_PTH")
        print(f"{self.__MODEL_TYPE} URL", VIT_H_PTH_URL)
        os.makedirs(os.path.join(CONFIG_PATH, self.__MODEL_TYPE), exist_ok=True)
        self.download_path = os.path.join(CONFIG_PATH, self.__MODEL_TYPE, self.__CHECKPOINT)
        if not os.path.exists(self.download_path):
            response = requests.get(VIT_H_PTH_URL)
            if response.status_code == 200:
                with open(self.download_path, 'wb') as file:
                    file.write(response.content)
                print(f"File downloaded successfully as {self.__CHECKPOINT}")
            else:
                print(f"Failed to download file. HTTP Status code: {response.status_code}")


    def convert_image_to_npArray(self, file_object: bytes) -> np.ndarray:
        """
        Automatically generate np array ob the binary image object
        Args:
            file_object (binary obj): file
        Returns:
            list: List of NumPy arrays.
        """
        np_array = np.asarray(bytearray(file_object), dtype="uint8")
        self.image = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
        if self.image is None:
            raise ValueError("Failed to decode the image. Please check the input file format.")
        return self.image

    def auto_annotate(self) -> list:
        """
        Automatically generate segmentation masks for an image.
        Args:
            image (np.ndarray): Input image as a NumPy array (H, W, C).
        Returns:
            list: List of segmentation masks as NumPy arrays.
        """
        masks = self.mask_generator.generate(self.image)
        return masks


    def annotate_with_box(self, box: list) -> np.ndarray:
        """
        Generate a segmentation mask for a given bounding box.
        Args:
            image (np.ndarray): Input image as a NumPy array (H, W, C).
            box (list): Bounding box [x1, y1, x2, y2].
        Returns:
            np.ndarray: Segmentation mask as a NumPy array.
        """
        self.predictor.set_image(self.image)
        mask, _, _ = self.predictor.predict(box=box)
        return mask

    def annotate_with_points(self, points: list, labels: list) -> np.ndarray:
        """
        Generate a segmentation mask for given points and labels.
        Args:
            image (np.ndarray): Input image as a NumPy array (H, W, C).
            points (list): List of points as (x, y) coordinates.
            labels (list): List of labels (1 for foreground, 0 for background).
        Returns:
            np.ndarray: Segmentation mask as a NumPy array.
        """
        self.predictor.set_image(self.image)
        mask, _, _ = self.predictor.predict(points=points, labels=labels)
        return mask

    def save_mask(self, mask: np.ndarray, filepath: str):
        """
        Save a segmentation mask to a file.
        Args:
            mask (np.ndarray): Segmentation mask as a NumPy array.
            filepath (str): File path to save the mask.
        """
        cv2.imwrite(filepath, mask.astype(np.uint8) * 255)

In [37]:
sam = SAM2Annotator()

MODEL_TYPE vit_h
CHECKPOINT sam_vit_h_4b8939.pth
vit_h URL https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


In [38]:
first_image = "Road621.jpg"
image_path = os.path.join(BASE_DIR, first_image)
img_arr = None

In [39]:
with open(image_path, 'rb') as file:
    file_object = file.read()
    print(type(file_object))
    img_arr = sam.convert_image_to_npArray(file_object=file_object)
    print(img_arr)
    print(sam.auto_annotate())

<class 'bytes'>
[[[236 237 228]
  [235 236 227]
  [232 235 226]
  ...
  [ 99 110 108]
  [121 132 130]
  [156 168 168]]

 [[173 170 162]
  [171 169 161]
  [169 167 159]
  ...
  [ 75  86  84]
  [ 92 103 101]
  [ 76  88  88]]

 [[155 144 140]
  [154 145 141]
  [155 146 142]
  ...
  [116 127 125]
  [144 155 153]
  [ 95 107 107]]

 ...

 [[ 88  76  70]
  [ 86  74  68]
  [ 85  73  67]
  ...
  [144 203 219]
  [144 204 220]
  [144 204 220]]

 [[ 93  81  75]
  [ 91  79  73]
  [ 90  78  72]
  ...
  [141 203 221]
  [140 204 222]
  [140 204 222]]

 [[ 99  87  81]
  [ 98  86  80]
  [ 96  84  78]
  ...
  [139 203 221]
  [138 204 222]
  [136 205 222]]]
[{'segmentation': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, Fal